In [4]:
# Smart Farm AI - Complete Crop Yield Prediction System for Google Colab
# ====================================================================
# Ready-to-run version optimized for Google Colab
# Author: Smart Farm AI Team
# Version: 2.1 (Enhanced with State Input and Map Integration)

# ============================================================================
# CELL 1: Install Required Libraries
# ============================================================================

!pip install pandas numpy matplotlib seaborn scikit-learn requests plotly folium -q
!pip install ipywidgets -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import requests
import json
from datetime import datetime, timedelta
import warnings
import joblib
import os
from typing import Dict, List, Optional

# For interactive widgets
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False

# For map visualization
try:
    import folium
    FOLIUM_AVAILABLE = True
except ImportError:
    FOLIUM_AVAILABLE = False

warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ All libraries installed and imported successfully!")
print("🌾 Smart Farm AI - Crop Yield Prediction System")
print("🚀 Ready to predict crop yields using AI and real-time data!")

# ============================================================================
# CELL 2: Configuration and Constants
# ============================================================================

# API Configuration (replace with your actual API key)
WEATHER_API_KEY = "4892543a92ba66aab0d480f6b5095df5"  # Replace with your OpenWeatherMap API key
WEATHER_BASE_URL = "https://api.openweathermap.org/data/2.5"

# Crop Information Database
CROP_INFO = {
    'Rice': {
        'optimal_temp': (20, 35),
        'optimal_ph': (5.5, 7.0),
        'water_requirement': 'high',
        'growth_period': 120,
        'optimal_rainfall': (1000, 2000),
        'base_yield': 4.5
    },
    'Wheat': {
        'optimal_temp': (15, 25),
        'optimal_ph': (6.0, 7.5),
        'water_requirement': 'medium',
        'growth_period': 150,
        'optimal_rainfall': (400, 800),
        'base_yield': 3.2
    },
    'Corn': {
        'optimal_temp': (18, 32),
        'optimal_ph': (6.0, 7.0),
        'water_requirement': 'medium',
        'growth_period': 100,
        'optimal_rainfall': (600, 1200),
        'base_yield': 6.8
    },
    'Soybean': {
        'optimal_temp': (20, 30),
        'optimal_ph': (6.0, 7.0),
        'water_requirement': 'medium',
        'growth_period': 110,
        'optimal_rainfall': (500, 1000),
        'base_yield': 2.4
    },
    'Cotton': {
        'optimal_temp': (21, 35),
        'optimal_ph': (5.8, 8.0),
        'water_requirement': 'high',
        'growth_period': 180,
        'optimal_rainfall': (600, 1200),
        'base_yield': 1.8
    }
}

# Regional information for India
REGIONS = {
    'North': {'temp_adj': 0, 'rainfall_adj': 1.0, 'soil_adj': 1.0},
    'South': {'temp_adj': 5, 'rainfall_adj': 1.2, 'soil_adj': 0.9},
    'East': {'temp_adj': 2, 'rainfall_adj': 1.5, 'soil_adj': 1.1},
    'West': {'temp_adj': -2, 'rainfall_adj': 0.7, 'soil_adj': 0.8},
    'Central': {'temp_adj': 1, 'rainfall_adj': 1.0, 'soil_adj': 1.0}
}

# New: Indian States Mapping (approximate lat/lon for state capitals and region)
STATES = {
    'Punjab': {'lat': 31.1471, 'lon': 75.3412, 'region': 'North'},
    'Haryana': {'lat': 30.7333, 'lon': 76.7794, 'region': 'North'},
    'Uttar Pradesh': {'lat': 26.8467, 'lon': 80.9462, 'region': 'Central'},
    'Rajasthan': {'lat': 27.0238, 'lon': 74.2179, 'region': 'West'},
    'Gujarat': {'lat': 23.0225, 'lon': 72.5714, 'region': 'West'},
    'Maharashtra': {'lat': 19.0760, 'lon': 72.8777, 'region': 'West'},
    'Madhya Pradesh': {'lat': 23.2599, 'lon': 77.4126, 'region': 'Central'},
    'Bihar': {'lat': 25.5941, 'lon': 85.1376, 'region': 'East'},
    'West Bengal': {'lat': 22.5726, 'lon': 88.3639, 'region': 'East'},
    'Odisha': {'lat': 20.2961, 'lon': 85.8245, 'region': 'East'},
    'Andhra Pradesh': {'lat': 15.9129, 'lon': 79.7400, 'region': 'South'},
    'Karnataka': {'lat': 12.9716, 'lon': 77.5946, 'region': 'South'},
    'Tamil Nadu': {'lat': 13.0827, 'lon': 80.2707, 'region': 'South'},
    'Kerala': {'lat': 10.8505, 'lon': 76.2711, 'region': 'South'},
    'Telangana': {'lat': 17.3850, 'lon': 78.4867, 'region': 'South'}
}

print("⚙️ Configuration loaded successfully!")
print(f"🌱 Supported crops: {', '.join(CROP_INFO.keys())}")
print(f"🌍 Supported regions: {', '.join(REGIONS.keys())}")
print(f"🏛️ Supported states: {', '.join(STATES.keys())}")

# ============================================================================
# CELL 3: Weather Data Integration Class
# ============================================================================

class WeatherDataIntegrator:
    """Handles real-time weather data integration"""

    def __init__(self, api_key: str = None):
        self.api_key = api_key or WEATHER_API_KEY
        self.base_url = WEATHER_BASE_URL

    def get_current_weather(self, lat: float, lon: float) -> Dict:
        """Fetch current weather data for given coordinates"""
        try:
            if self.api_key and self.api_key != "YOUR_OPENWEATHER_API_KEY":
                url = f"{self.base_url}/weather"
                params = {
                    'lat': lat,
                    'lon': lon,
                    'appid': self.api_key,
                    'units': 'metric'
                }

                response = requests.get(url, params=params, timeout=10)

                if response.status_code == 200:
                    data = response.json()
                    return {
                        'temperature': data['main']['temp'],
                        'humidity': data['main']['humidity'],
                        'pressure': data['main']['pressure'],
                        'wind_speed': data['wind']['speed'],
                        'weather_condition': data['weather'][0]['description'],
                        'timestamp': datetime.now().isoformat(),
                        'source': 'OpenWeatherMap'
                    }

            # Fallback to mock data
            return self._generate_mock_weather_data()

        except Exception as e:
            print(f"⚠️ Weather API error: {e}")
            return self._generate_mock_weather_data()

    def get_weather_forecast(self, lat: float, lon: float, days: int = 7) -> List[Dict]:
        """Fetch weather forecast for upcoming days"""
        try:
            if self.api_key and self.api_key != "YOUR_OPENWEATHER_API_KEY":
                url = f"{self.base_url}/forecast"
                params = {
                    'lat': lat,
                    'lon': lon,
                    'appid': self.api_key,
                    'units': 'metric',
                    'cnt': days * 8  # 3-hour intervals
                }

                response = requests.get(url, params=params, timeout=10)

                if response.status_code == 200:
                    data = response.json()
                    forecast_data = []

                    for item in data['list'][::8]:  # Take one per day
                        forecast_data.append({
                            'date': datetime.fromtimestamp(item['dt']).date().isoformat(),
                            'temperature': item['main']['temp'],
                            'humidity': item['main']['humidity'],
                            'precipitation': item.get('rain', {}).get('3h', 0),
                            'weather_condition': item['weather'][0]['description']
                        })

                    return forecast_data

            # Fallback to mock data
            return self._generate_mock_forecast_data(days)

        except Exception as e:
            print(f"⚠️ Forecast API error: {e}")
            return self._generate_mock_forecast_data(days)

    def _generate_mock_weather_data(self) -> Dict:
        """Generate realistic mock current weather data"""
        return {
            'temperature': round(np.random.uniform(15, 35), 1),
            'humidity': round(np.random.uniform(40, 90)),
            'pressure': round(np.random.uniform(980, 1030)),
            'wind_speed': round(np.random.uniform(0, 15), 1),
            'weather_condition': np.random.choice([
                'clear sky', 'few clouds', 'scattered clouds',
                'broken clouds', 'light rain', 'moderate rain'
            ]),
            'timestamp': datetime.now().isoformat(),
            'source': 'Mock Data'
        }

    def _generate_mock_forecast_data(self, days: int) -> List[Dict]:
        """Generate realistic mock forecast data"""
        forecast_data = []
        base_date = datetime.now().date()
        base_temp = np.random.uniform(20, 30)

        for i in range(days):
            date = base_date + timedelta(days=i)
            temp_variation = np.random.uniform(-5, 5)
            temp = max(10, min(40, base_temp + temp_variation))

            forecast_data.append({
                'date': date.isoformat(),
                'temperature': round(temp, 1),
                'humidity': round(np.random.uniform(40, 90)),
                'precipitation': max(0, round(np.random.normal(5, 10), 1)),
                'weather_condition': np.random.choice([
                    'clear sky', 'few clouds', 'scattered clouds', 'light rain'
                ])
            })

        return forecast_data

print("🌤️ Weather integration class created successfully!")

# ============================================================================
# CELL 4: Soil Data Integration Class
# ============================================================================

class SoilDataIntegrator:
    """Handles soil health data integration and analysis"""

    def __init__(self):
        self.soil_properties = [
            'ph', 'nitrogen', 'phosphorus', 'potassium',
            'organic_matter', 'clay_content', 'sand_content', 'silt_content'
        ]

    def get_soil_analysis(self, lat: float, lon: float) -> Dict:
        """Get comprehensive soil analysis with health score and recommendations"""
        soil_data = self._generate_realistic_soil_data(lat, lon)
        health_score = self._calculate_soil_health_score(soil_data)
        recommendations = self._generate_soil_recommendations(soil_data)

        analysis = {
            'soil_data': soil_data,
            'health_score': health_score,
            'recommendations': recommendations,
            'analysis_date': datetime.now().isoformat()
        }

        return analysis

    def _generate_realistic_soil_data(self, lat: float, lon: float) -> Dict:
        """Generate realistic soil data based on geographic location"""
        lat_factor = abs(lat) / 90.0

        # Adjust soil properties based on climate zones
        base_ph = 6.5 - lat_factor * 1.5 + np.random.uniform(-0.5, 0.5)
        base_ph = max(4.5, min(8.5, base_ph))

        base_om = 3.0 - lat_factor * 1.0 + np.random.uniform(-1.0, 1.0)
        base_om = max(0.5, min(6.0, base_om))

        return {
            'ph': round(base_ph, 2),
            'nitrogen': round(np.random.uniform(50, 300)),
            'phosphorus': round(np.random.uniform(10, 80)),
            'potassium': round(np.random.uniform(100, 400)),
            'organic_matter': round(base_om, 2),
            'clay_content': round(np.random.uniform(15, 45)),
            'sand_content': round(np.random.uniform(25, 65)),
            'silt_content': round(np.random.uniform(10, 35)),
            'moisture_content': round(np.random.uniform(15, 35)),
            'salinity': round(np.random.uniform(0.1, 2.0), 2),
            'timestamp': datetime.now().isoformat(),
            'source': 'Simulated based on location'
        }

    def _calculate_soil_health_score(self, soil_data: Dict) -> float:
        """Calculate comprehensive soil health score (0-100)"""
        score = 0

        # pH score (25 points)
        ph = soil_data['ph']
        if 6.0 <= ph <= 7.5:
            ph_score = 25
        else:
            ph_score = max(0, 25 - abs(ph - 6.75) * 8)
        score += ph_score

        # Organic matter score (25 points)
        om = soil_data['organic_matter']
        om_score = min(25, om * 8)
        score += om_score

        # Nitrogen score (20 points)
        nitrogen_score = min(20, soil_data['nitrogen'] / 10)
        score += nitrogen_score

        # Phosphorus score (15 points)
        phosphorus_score = min(15, soil_data['phosphorus'] * 0.6)
        score += phosphorus_score

        # Potassium score (15 points)
        potassium_score = min(15, soil_data['potassium'] / 15)
        score += potassium_score

        return round(min(100, score), 1)

    def _generate_soil_recommendations(self, soil_data: Dict) -> List[str]:
        """Generate actionable soil management recommendations"""
        recommendations = []

        # pH recommendations
        ph = soil_data['ph']
        if ph < 5.5:
            recommendations.append("🧪 Soil is very acidic. Apply 2-3 tons of lime per hectare")
        elif ph < 6.0:
            recommendations.append("🧪 Soil is acidic. Apply 1-2 tons of lime per hectare")
        elif ph > 8.0:
            recommendations.append("🧪 Soil is alkaline. Apply sulfur or organic matter to lower pH")

        # Nutrient recommendations
        if soil_data['nitrogen'] < 100:
            recommendations.append("🌱 Low nitrogen levels. Apply nitrogen fertilizer (urea 150-200 kg/ha)")
        if soil_data['phosphorus'] < 20:
            recommendations.append("💎 Low phosphorus levels. Apply DAP or SSP (100-150 kg/ha)")
        if soil_data['potassium'] < 150:
            recommendations.append("⚡ Low potassium levels. Apply muriate of potash (50-100 kg/ha)")

        # Organic matter recommendations
        om = soil_data['organic_matter']
        if om < 1.5:
            recommendations.append("🍂 Very low organic matter. Add 5-10 tons of compost per hectare")
        elif om < 2.5:
            recommendations.append("🍂 Low organic matter. Add 3-5 tons of organic matter per hectare")

        return recommendations

print("🌍 Soil integration class created successfully!")

# ============================================================================
# CELL 5: Complete Crop Yield Prediction System
# ============================================================================

class CropYieldPredictionSystem:
    """Complete AI-based crop yield prediction system"""

    def __init__(self, weather_api_key: str = None):
        self.models = {}
        self.scaler = StandardScaler()
        self.label_encoders = {}
        self.feature_names = []
        self.is_trained = False

        # Initialize data integrators
        self.weather_integrator = WeatherDataIntegrator(weather_api_key)
        self.soil_integrator = SoilDataIntegrator()

        print("🚀 Crop Yield Prediction System initialized!")

    def generate_training_data(self, n_samples: int = 5000) -> pd.DataFrame:
        """Generate comprehensive synthetic training data"""
        print(f"📊 Generating {n_samples:,} training samples...")

        np.random.seed(42)

        # Basic features
        crops = list(CROP_INFO.keys())
        regions = list(REGIONS.keys())

        data = {
            'crop_type': np.random.choice(crops, n_samples),
            'region': np.random.choice(regions, n_samples),
            'year': np.random.randint(2015, 2024, n_samples),
            'area_hectares': np.random.uniform(0.5, 50, n_samples),
        }

        # Weather features
        seasons = np.random.uniform(0, 1, n_samples)
        base_temp = 20 + seasons * 15 + np.random.normal(0, 5, n_samples)
        data.update({
            'avg_temperature': np.clip(base_temp, 5, 45),
            'total_rainfall': np.random.exponential(800, n_samples),
            'humidity': np.random.uniform(30, 95, n_samples),
            'sunshine_hours': np.random.uniform(1200, 3200, n_samples),
        })

        # Soil features
        base_ph = np.random.normal(6.5, 1.0, n_samples)
        data.update({
            'soil_ph': np.clip(base_ph, 4.0, 9.0),
            'soil_nitrogen': np.random.uniform(30, 350, n_samples),
            'soil_phosphorus': np.random.uniform(8, 100, n_samples),
            'soil_potassium': np.random.uniform(80, 500, n_samples),
            'organic_matter': np.random.uniform(0.5, 7, n_samples),
        })

        # Management practices
        data.update({
            'irrigation_frequency': np.random.randint(0, 25, n_samples),
            'fertilizer_amount': np.random.uniform(0, 600, n_samples),
            'pesticide_usage': np.random.uniform(0, 12, n_samples),
        })

        # Calculate realistic yield
        yields = []
        for i in range(n_samples):
            crop_type = data['crop_type'][i]
            region = data['region'][i]

            # Base yield from crop type
            base_yield = CROP_INFO[crop_type]['base_yield']

            # Regional adjustment
            regional_adj = REGIONS[region]['soil_adj']

            # Weather impact
            temp = data['avg_temperature'][i]
            optimal_temp = CROP_INFO[crop_type]['optimal_temp']
            temp_factor = 1.0
            if temp < optimal_temp[0]:
                temp_factor = 0.7 + (temp - 10) / (optimal_temp[0] - 10) * 0.3
            elif temp > optimal_temp[1]:
                temp_factor = 1.0 - (temp - optimal_temp[1]) / 20 * 0.4
            temp_factor = max(0.3, min(1.2, temp_factor))

            # Rainfall impact
            rainfall = data['total_rainfall'][i]
            optimal_rainfall = CROP_INFO[crop_type]['optimal_rainfall']
            if optimal_rainfall[0] <= rainfall <= optimal_rainfall[1]:
                rain_factor = 1.0
            elif rainfall < optimal_rainfall[0]:
                rain_factor = 0.5 + (rainfall / optimal_rainfall[0]) * 0.5
            else:
                rain_factor = 1.0 - min(0.4, (rainfall - optimal_rainfall[1]) / rainfall * 0.8)
            rain_factor = max(0.2, min(1.3, rain_factor))

            # Soil impact
            ph = data['soil_ph'][i]
            optimal_ph = CROP_INFO[crop_type]['optimal_ph']
            if optimal_ph[0] <= ph <= optimal_ph[1]:
                ph_factor = 1.0
            else:
                ph_factor = max(0.6, 1.0 - abs(ph - np.mean(optimal_ph)) * 0.15)

            # Nutrient impact
            nutrient_score = (data['soil_nitrogen'][i]/200 +
                            data['soil_phosphorus'][i]/50 +
                            data['soil_potassium'][i]/300 +
                            data['organic_matter'][i]/4) / 4
            nutrient_factor = 0.5 + nutrient_score * 0.7
            nutrient_factor = max(0.4, min(1.4, nutrient_factor))

            # Management impact
            irrigation_factor = min(1.3, 0.8 + data['irrigation_frequency'][i]/20)
            fertilizer_factor = min(1.25, 0.9 + data['fertilizer_amount'][i]/400)
            pesticide_factor = max(0.85, 1.1 - data['pesticide_usage'][i]/15)

            # Calculate final yield per hectare
            yield_per_ha = (base_yield * regional_adj * temp_factor * rain_factor *
                          ph_factor * nutrient_factor * irrigation_factor *
                          fertilizer_factor * pesticide_factor *
                          np.random.uniform(0.8, 1.2))

            # Total yield
            total_yield = yield_per_ha * data['area_hectares'][i]
            yields.append(max(0.1, total_yield))

        data['yield_tonnes'] = yields
        df = pd.DataFrame(data)

        print(f"✅ Generated dataset with {df.shape[0]:,} samples")
        print(f"📈 Yield range: {df['yield_tonnes'].min():.1f} - {df['yield_tonnes'].max():.1f} tonnes")

        return df

    def preprocess_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Preprocess data for machine learning"""
        print("🔄 Preprocessing data...")

        # Handle categorical variables
        categorical_features = ['crop_type', 'region']
        for feature in categorical_features:
            if feature not in self.label_encoders:
                self.label_encoders[feature] = LabelEncoder()
                df[feature + '_encoded'] = self.label_encoders[feature].fit_transform(df[feature])
            else:
                df[feature + '_encoded'] = self.label_encoders[feature].transform(df[feature])

        # Feature engineering
        df['yield_per_hectare'] = df['yield_tonnes'] / df['area_hectares']
        df['rainfall_per_temp'] = df['total_rainfall'] / (df['avg_temperature'] + 1)
        df['nutrient_index'] = (df['soil_nitrogen'] + df['soil_phosphorus'] + df['soil_potassium']) / 3
        df['management_score'] = (df['irrigation_frequency'] + df['fertilizer_amount']/50 - df['pesticide_usage']) / 3
        df['soil_texture'] = df['organic_matter'] * df['soil_ph']

        # Select features for modeling
        self.feature_names = [
            'crop_type_encoded', 'region_encoded', 'year', 'area_hectares',
            'avg_temperature', 'total_rainfall', 'humidity', 'sunshine_hours',
            'soil_ph', 'soil_nitrogen', 'soil_phosphorus', 'soil_potassium', 'organic_matter',
            'irrigation_frequency', 'fertilizer_amount', 'pesticide_usage',
            'rainfall_per_temp', 'nutrient_index', 'management_score', 'soil_texture'
        ]

        processed_df = df[self.feature_names + ['yield_tonnes']].copy()

        print(f"✅ Data preprocessing complete. Features: {len(self.feature_names)}")
        return processed_df

    def train_models(self, df: pd.DataFrame) -> Dict:
        """Train multiple ML models and return performance metrics"""
        print("🤖 Training machine learning models...")

        # Prepare data
        X = df[self.feature_names]
        y = df['yield_tonnes']

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        # Define models
        models_config = {
            'Random Forest': {
                'model': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42),
                'scaled': False
            },
            'Gradient Boosting': {
                'model': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
                'scaled': False
            },
            'Neural Network': {
                'model': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42),
                'scaled': True
            }
        }

        results = {}

        for name, config in models_config.items():
            print(f"Training {name}...")

            model = config['model']

            if config['scaled']:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
            else:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

            # Calculate metrics
            mse = mean_squared_error(y_test, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

            results[name] = {
                'model': model,
                'rmse': rmse,
                'mae': mae,
                'r2': r2,
                'mape': mape,
                'scaled': config['scaled']
            }

            print(f"  ✅ {name}: R²={r2:.3f}, RMSE={rmse:.2f}, MAE={mae:.2f}")

        self.models = results
        self.is_trained = True

        # Select best model
        best_model_name = max(results.keys(), key=lambda k: results[k]['r2'])
        print(f"🏆 Best model: {best_model_name} (R² = {results[best_model_name]['r2']:.3f})")

        return results

    def predict_yield(self, lat: float, lon: float, crop_type: str, area_hectares: float,
                     irrigation_freq: int = 10, fertilizer_amount: float = 200,
                     pesticide_usage: float = 3, model_name: str = None) -> Dict:
        """Make crop yield prediction with real-time data integration"""

        if not self.is_trained:
            raise ValueError("Model not trained yet. Please run train_models() first.")

        print(f"🔮 Predicting yield for {crop_type} at ({lat:.4f}, {lon:.4f})...")

        # Get real-time data
        current_weather = self.weather_integrator.get_current_weather(lat, lon)
        soil_analysis = self.soil_integrator.get_soil_analysis(lat, lon)
        weather_forecast = self.weather_integrator.get_weather_forecast(lat, lon, days=7)

        # Determine region
        region = self._get_region_from_coordinates(lat, lon)

        # Prepare features
        prediction_features = self._prepare_prediction_features(
            current_weather, soil_analysis, weather_forecast,
            crop_type, region, area_hectares, irrigation_freq,
            fertilizer_amount, pesticide_usage
        )

        # Select model
        if model_name is None:
            model_name = max(self.models.keys(), key=lambda k: self.models[k]['r2'])

        model_info = self.models[model_name]
        model = model_info['model']

        # Create feature vector
        feature_vector = []
        for feature_name in self.feature_names:
            if feature_name in prediction_features:
                feature_vector.append(prediction_features[feature_name])
            else:
                feature_vector.append(0)

        feature_vector = np.array(feature_vector).reshape(1, -1)

        # Make prediction
        if model_info['scaled']:
            feature_vector = self.scaler.transform(feature_vector)

        predicted_yield = model.predict(feature_vector)[0]
        yield_per_hectare = predicted_yield / area_hectares

        # Calculate confidence intervals
        model_rmse = model_info['rmse']
        confidence_interval = {
            'lower': max(0, predicted_yield - 1.96 * model_rmse),
            'upper': predicted_yield + 1.96 * model_rmse
        }

        result = {
            'prediction': {
                'total_yield': round(predicted_yield, 2),
                'yield_per_hectare': round(yield_per_hectare, 2),
                'confidence_interval': confidence_interval,
                'model_used': model_name,
                'model_accuracy': round(model_info['r2'], 3)
            },
            'field_data': {
                'weather': current_weather,
                'soil': soil_analysis,
                'forecast': weather_forecast
            },
            'input_parameters': {
                'location': {'latitude': lat, 'longitude': lon, 'region': region},
                'crop_type': crop_type,
                'area_hectares': area_hectares,
                'irrigation_frequency': irrigation_freq,
                'fertilizer_amount': fertilizer_amount,
                'pesticide_usage': pesticide_usage
            },
            'timestamp': datetime.now().isoformat()
        }

        print(f"✅ Prediction: {predicted_yield:.2f} tonnes ({yield_per_hectare:.2f} t/ha)")
        return result

    def _get_region_from_coordinates(self, lat: float, lon: float) -> str:
        """Determine region based on coordinates (simplified for India)"""
        if lat > 30:
            return 'North'
        elif lat < 15:
            return 'South'
        elif lon > 85:
            return 'East'
        elif lon < 75:
            return 'West'
        else:
            return 'Central'

    def _prepare_prediction_features(self, weather: Dict, soil_analysis: Dict,
                                   forecast: List[Dict], crop_type: str, region: str,
                                   area_hectares: float, irrigation_freq: int,
                                   fertilizer_amount: float, pesticide_usage: float) -> Dict:
        """Prepare features for ML prediction"""

        soil_data = soil_analysis['soil_data']

        # Estimate annual rainfall from weekly forecast (simplified)
        weekly_rain = sum(day['precipitation'] for day in forecast)
        estimated_annual_rain = weekly_rain * 52

        features = {
            'crop_type': crop_type,
            'region': region,
            'year': datetime.now().year,
            'area_hectares': area_hectares,
            'avg_temperature': weather['temperature'],
            'total_rainfall': estimated_annual_rain,
            'humidity': weather['humidity'],
            'sunshine_hours': 2400,
            'soil_ph': soil_data['ph'],
            'soil_nitrogen': soil_data['nitrogen'],
            'soil_phosphorus': soil_data['phosphorus'],
            'soil_potassium': soil_data['potassium'],
            'organic_matter': soil_data['organic_matter'],
            'irrigation_frequency': irrigation_freq,
            'fertilizer_amount': fertilizer_amount,
            'pesticide_usage': pesticide_usage
        }

        # Encode categorical features
        if 'crop_type' in self.label_encoders:
            try:
                features['crop_type_encoded'] = self.label_encoders['crop_type'].transform([crop_type])[0]
            except ValueError:
                features['crop_type_encoded'] = 0

        if 'region' in self.label_encoders:
            try:
                features['region_encoded'] = self.label_encoders['region'].transform([region])[0]
            except ValueError:
                features['region_encoded'] = 0

        # Feature engineering
        features['rainfall_per_temp'] = features['total_rainfall'] / (features['avg_temperature'] + 1)
        features['nutrient_index'] = (features['soil_nitrogen'] + features['soil_phosphorus'] +
                                    features['soil_potassium']) / 3
        features['management_score'] = (features['irrigation_frequency'] +
                                      features['fertilizer_amount']/50 -
                                      features['pesticide_usage']) / 3
        features['soil_texture'] = features['organic_matter'] * features['soil_ph']

        return features

    def generate_recommendations(self, prediction_result: Dict) -> Dict:
        """Generate comprehensive recommendations based on prediction and field conditions"""

        crop_type = prediction_result['input_parameters']['crop_type']
        weather = prediction_result['field_data']['weather']
        soil_analysis = prediction_result['field_data']['soil']
        forecast = prediction_result['field_data']['forecast']

        recommendations = {
            'irrigation': [],
            'fertilization': [],
            'pest_control': [],
            'general': []
        }

        # Get optimal conditions for the crop
        if crop_type in CROP_INFO:
            optimal = CROP_INFO[crop_type]

            # Temperature-based recommendations
            current_temp = weather['temperature']
            optimal_temp = optimal['optimal_temp']

            if current_temp < optimal_temp[0] - 3:
                recommendations['general'].append(
                    f"Temperature ({current_temp}°C) is significantly below optimal for {crop_type}. Consider protective measures."
                )
            elif current_temp > optimal_temp[1] + 3:
                recommendations['irrigation'].append(
                    f"High temperature ({current_temp}°C) detected. Increase irrigation frequency."
                )

        # Soil-based recommendations
        soil_data = soil_analysis['soil_data']
        soil_ph = soil_data['ph']

        if soil_ph < 5.5:
            recommendations['fertilization'].append(f"Soil is very acidic (pH {soil_ph}). Apply lime at 2-3 tons/hectare.")
        elif soil_ph > 8.0:
            recommendations['fertilization'].append(f"Soil is alkaline (pH {soil_ph}). Apply sulfur or organic acids.")

        # Nutrient recommendations
        if soil_data['nitrogen'] < 100:
            recommendations['fertilization'].append(f"Nitrogen is low ({soil_data['nitrogen']} ppm). Apply urea at 150-200 kg/hectare.")
        if soil_data['phosphorus'] < 20:
            recommendations['fertilization'].append(f"Phosphorus is low ({soil_data['phosphorus']} ppm). Apply DAP at 100-150 kg/hectare.")
        if soil_data['potassium'] < 150:
            recommendations['fertilization'].append(f"Potassium is low ({soil_data['potassium']} ppm). Apply MOP at 50-100 kg/hectare.")

        # Weather forecast recommendations
        upcoming_rain = sum(day['precipitation'] for day in forecast[:3])
        avg_humidity = np.mean([day['humidity'] for day in forecast[:3]])

        if upcoming_rain < 5:
            recommendations['irrigation'].append("Dry weather expected. Plan irrigation for next 3 days.")
        elif upcoming_rain > 25:
            recommendations['irrigation'].append("Heavy rainfall expected. Ensure proper drainage.")

        if avg_humidity > 80:
            recommendations['pest_control'].append("High humidity forecasted. Monitor for fungal diseases.")

        # Add soil analysis recommendations
        recommendations['general'].extend(soil_analysis['recommendations'])

        return recommendations

    def visualize_results(self, prediction_result: Dict, recommendations: Dict = None):
        """Create comprehensive visualizations for the prediction results"""

        if recommendations is None:
            recommendations = self.generate_recommendations(prediction_result)

        fig = plt.figure(figsize=(20, 12))
        gs = fig.add_gridspec(2, 4, hspace=0.3, wspace=0.3)

        # 1. Yield Prediction
        ax1 = fig.add_subplot(gs[0, 0])
        crop_type = prediction_result['input_parameters']['crop_type']
        total_yield = prediction_result['prediction']['total_yield']
        yield_per_ha = prediction_result['prediction']['yield_per_hectare']

        bars = ax1.bar(['Predicted\nYield'], [total_yield], color='lightgreen', alpha=0.8)
        ax1.set_ylabel('Yield (tonnes)')
        ax1.set_title(f'{crop_type} Yield Prediction\n{total_yield:.2f} tonnes ({yield_per_ha:.2f} t/ha)', fontweight='bold')
        ax1.grid(True, alpha=0.3)

        # 2. Current Weather
        ax2 = fig.add_subplot(gs[0, 1])
        weather = prediction_result['field_data']['weather']
        weather_params = ['Temp\n(°C)', 'Humidity\n(%)', 'Wind\n(km/h)']
        weather_values = [weather['temperature'], weather['humidity'], weather['wind_speed']]

        bars2 = ax2.bar(weather_params, weather_values, color=['orange', 'skyblue', 'lightcoral'])
        ax2.set_title('Current Weather', fontweight='bold')
        ax2.grid(True, alpha=0.3)

        # 3. Soil Health
        ax3 = fig.add_subplot(gs[0, 2])
        soil_data = prediction_result['field_data']['soil']['soil_data']
        health_score = prediction_result['field_data']['soil']['health_score']

        sizes = [health_score, 100 - health_score]
        colors = ['lightgreen' if health_score > 70 else 'orange' if health_score > 50 else 'lightcoral', 'lightgray']
        ax3.pie(sizes, labels=['Healthy', 'Needs Care'], colors=colors, autopct='%1.1f%%', startangle=90)
        ax3.set_title(f'Soil Health: {health_score:.1f}/100', fontweight='bold')

        # 4. Soil Nutrients
        ax4 = fig.add_subplot(gs[0, 3])
        nutrients = ['N', 'P', 'K', 'OM']
        nutrient_values = [soil_data['nitrogen'], soil_data['phosphorus'],
                          soil_data['potassium'], soil_data['organic_matter']]

        bars4 = ax4.bar(nutrients, nutrient_values, color=['lightblue', 'lightcoral', 'lightgreen', 'wheat'])
        ax4.set_ylabel('Level')
        ax4.set_title('Soil Nutrients', fontweight='bold')
        ax4.grid(True, alpha=0.3)

        # 5. Weather Forecast
        ax5 = fig.add_subplot(gs[1, 0:2])
        forecast = prediction_result['field_data']['forecast']
        dates = [day['date'][-5:] for day in forecast]
        temps = [day['temperature'] for day in forecast]
        precip = [day['precipitation'] for day in forecast]

        ax5_twin = ax5.twinx()
        line1 = ax5.plot(dates, temps, 'ro-', label='Temperature (°C)', linewidth=2)
        bars5 = ax5_twin.bar(dates, precip, alpha=0.6, color='lightblue', label='Precipitation (mm)')

        ax5.set_ylabel('Temperature (°C)', color='red')
        ax5_twin.set_ylabel('Precipitation (mm)', color='blue')
        ax5.set_title('7-Day Weather Forecast', fontweight='bold')
        ax5.tick_params(axis='x', rotation=45)

        # 6. Recommendations Summary
        ax6 = fig.add_subplot(gs[1, 2:])
        rec_counts = {category: len(recs) for category, recs in recommendations.items()}
        total_recommendations = sum(rec_counts.values())

        if total_recommendations > 0:
            categories = list(rec_counts.keys())
            counts = list(rec_counts.values())
            colors_rec = ['lightcoral', 'lightgreen', 'lightsalmon', 'lightblue']

            bars6 = ax6.bar(categories, counts, color=colors_rec[:len(categories)])
            ax6.set_ylabel('Number of Recommendations')
            ax6.set_title(f'Smart Farming Recommendations ({total_recommendations} total)', fontweight='bold')
            ax6.grid(True, alpha=0.3)
            ax6.tick_params(axis='x', rotation=45)
        else:
            ax6.text(0.5, 0.5, 'No specific recommendations\nCurrent conditions are optimal',
                    ha='center', va='center', transform=ax6.transAxes,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.7))
            ax6.set_title('Smart Farming Recommendations', fontweight='bold')
            ax6.axis('off')

        # Overall title
        location = prediction_result['input_parameters']['location']
        fig.suptitle(f'Smart Farm AI - Crop Yield Analysis Report\n'
                    f'{crop_type} at {location["latitude"]:.4f}°N, {location["longitude"]:.4f}°E',
                    fontsize=16, fontweight='bold')

        plt.tight_layout()
        plt.show()

        # Print detailed summary
        self._print_summary(prediction_result, recommendations)

    def _print_summary(self, prediction_result: Dict, recommendations: Dict):
        """Print detailed text summary"""
        print("\n" + "="*60)
        print("SMART FARM AI - CROP YIELD ANALYSIS REPORT")
        print("="*60)

        inputs = prediction_result['input_parameters']
        prediction = prediction_result['prediction']

        print(f"Location: {inputs['location']['latitude']:.4f}°N, {inputs['location']['longitude']:.4f}°E")
        print(f"Region: {inputs['location']['region']}")
        print(f"Crop: {inputs['crop_type']}")
        print(f"Area: {inputs['area_hectares']} hectares")

        print(f"\nPREDICTION RESULTS")
        print("-"*30)
        print(f"Total Predicted Yield: {prediction['total_yield']:.2f} tonnes")
        print(f"Yield per Hectare: {prediction['yield_per_hectare']:.2f} tonnes/ha")
        print(f"Model Used: {prediction['model_used']} (R² = {prediction['model_accuracy']:.3f})")

        weather = prediction_result['field_data']['weather']
        soil = prediction_result['field_data']['soil']

        print(f"\nCURRENT CONDITIONS")
        print("-"*30)
        print(f"Temperature: {weather['temperature']:.1f}°C")
        print(f"Humidity: {weather['humidity']:.1f}%")
        print(f"Weather: {weather['weather_condition'].title()}")
        print(f"Soil Health Score: {soil['health_score']:.1f}/100")
        print(f"Soil pH: {soil['soil_data']['ph']:.1f}")

        print(f"\nRECOMMENDAITONS")
        print("-"*30)
        total_recs = sum(len(recs) for recs in recommendations.values())
        if total_recs == 0:
            print("Current conditions are optimal. No specific recommendations needed.")
        else:
            for category, recs in recommendations.items():
                if recs:
                    print(f"\n{category.upper()}:")
                    for i, rec in enumerate(recs, 1):
                        print(f"  {i}. {rec}")

        print("="*60)

    # New Feature: Generate and display interactive map
    def display_location_map(self, lat: float, lon: float, prediction_result: Dict):
        """Display folium map centered on location with prediction popup"""
        if not FOLIUM_AVAILABLE:
            print("⚠️ Folium not available. Install with !pip install folium")
            return

        # Create map centered on location
        m = folium.Map(location=[lat, lon], zoom_start=10)

        # Add marker with prediction info
        crop = prediction_result['input_parameters']['crop_type']
        yield_total = prediction_result['prediction']['total_yield']
        yield_ha = prediction_result['prediction']['yield_per_hectare']
        soil_score = prediction_result['field_data']['soil']['health_score']
        temp = prediction_result['field_data']['weather']['temperature']

        popup_html = f"""
        <div style="width: 200px;">
            <h4>🌾 {crop} Farm</h4>
            <p><b>Predicted Yield:</b> {yield_total:.2f} tonnes</p>
            <p><b>Per Hectare:</b> {yield_ha:.2f} t/ha</p>
            <p><b>Soil Health:</b> {soil_score:.1f}/100</p>
            <p><b>Current Temp:</b> {temp:.1f}°C</p>
            <p><i>Smart Farm AI Prediction</i></p>
        </div>
        """

        folium.Marker(
            [lat, lon],
            popup=folium.Popup(popup_html, max_width=250),
            tooltip=f"{crop} Yield: {yield_total:.2f} tonnes",
            icon=folium.Icon(color='green', icon='cloud-sun')
        ).add_to(m)

        # Add a circle for farm area (approximate)
        area_ha = prediction_result['input_parameters']['area_hectares']
        radius_m = np.sqrt(area_ha * 10000 / np.pi)  # Approximate radius in meters
        folium.Circle(
            [lat, lon],
            radius=radius_m,
            popup=f"Farm Area: {area_ha} ha",
            color='blue',
            fill=False
        ).add_to(m)

        display(m)
        print("🗺️ Interactive map displayed! Zoom and pan to explore.")

print("Complete Crop Yield Prediction System class created!")

# ============================================================================
# CELL 6: Quick Start Functions
# ============================================================================

def quick_predict(lat=28.6139, lon=77.2090, crop='Rice', area=5.0, state=None):
    """Quick prediction function for immediate results (now supports state input)"""

    # New: Handle state input
    if state and state in STATES:
        lat = STATES[state]['lat']
        lon = STATES[state]['lon']
        print(f"📍 Using state '{state}': Setting location to ({lat:.4f}, {lon:.4f})")

    print(f"Quick prediction for {crop} at {lat:.4f}°N, {lon:.4f}°E")

    # Create system
    system = CropYieldPredictionSystem()

    # Generate training data
    df = system.generate_training_data(3000)
    processed_df = system.preprocess_data(df)

    # Train models
    results = system.train_models(processed_df)

    # Make prediction
    result = system.predict_yield(lat, lon, crop, area)

    # Generate recommendations
    recommendations = system.generate_recommendations(result)

    # Visualize
    system.visualize_results(result, recommendations)

    # New: Display map
    system.display_location_map(lat, lon, result)

    return result

def run_complete_demo():
    """Run comprehensive demonstration"""

    print("Starting Smart Farm AI demonstration...")
    print("="*50)

    # Initialize system
    prediction_system = CropYieldPredictionSystem()

    # Generate and train
    print("\n1. Generating training data...")
    df = prediction_system.generate_training_data(5000)

    print("\n2. Preprocessing data...")
    processed_df = prediction_system.preprocess_data(df)

    print("\n3. Training models...")
    results = prediction_system.train_models(processed_df)

    # Show model performance
    print("\nModel Performance:")
    for name, result in results.items():
        print(f"{name}: R²={result['r2']:.3f}, RMSE={result['rmse']:.2f}")

    # Example predictions
    print("\n4. Making sample predictions...")

    # Rice in Punjab
    print("\nExample 1: Rice in Punjab, India")
    result1 = prediction_system.predict_yield(
        lat=30.7333, lon=76.7794, crop_type='Rice', area_hectares=10.0,
        irrigation_freq=15, fertilizer_amount=300, pesticide_usage=2
    )

    recommendations1 = prediction_system.generate_recommendations(result1)
    prediction_system.visualize_results(result1, recommendations1)
    prediction_system.display_location_map(30.7333, 76.7794, result1)

    # Wheat in UP
    print("\nExample 2: Wheat in Uttar Pradesh, India")
    result2 = prediction_system.predict_yield(
        lat=26.8467, lon=80.9462, crop_type='Wheat', area_hectares=5.0,
        irrigation_freq=8, fertilizer_amount=200, pesticide_usage=3
    )

    recommendations2 = prediction_system.generate_recommendations(result2)
    prediction_system.visualize_results(result2, recommendations2)
    prediction_system.display_location_map(26.8467, 80.9462, result2)

    print("\nDemo completed successfully!")
    return prediction_system

# ============================================================================
# CELL 7: Interactive Interface (if widgets available)
# ============================================================================

if WIDGETS_AVAILABLE:
    def create_interactive_predictor():
        """Create interactive interface for predictions (enhanced with state/coord input)"""

        # New: Input method selector
        input_method = widgets.RadioButtons(
            options=[('Select by State', 'state'), ('Enter Coordinates', 'coord')],
            value='state',
            description='Input Method:',
            style={'description_width': 'initial'}
        )

        # State dropdown
        state_widget = widgets.Dropdown(
            options=list(STATES.keys()),
            value='Punjab',
            description='State:',
            disabled=False
        )

        # Coordinate inputs (initially disabled)
        lat_widget = widgets.FloatText(value=31.1471, description='Latitude:', step=0.0001, disabled=True)
        lon_widget = widgets.FloatText(value=75.3412, description='Longitude:', step=0.0001, disabled=True)

        # Link state to coords
        def update_coords(change):
            if change['type'] == 'change' and change['name'] == 'value':
                selected_state = state_widget.value
                if selected_state in STATES:
                    lat_widget.value = STATES[selected_state]['lat']
                    lon_widget.value = STATES[selected_state]['lon']

        state_widget.observe(update_coords, names='value')

        # Toggle inputs based on method
        def toggle_inputs(change):
            if input_method.value == 'state':
                state_widget.disabled = False
                lat_widget.disabled = True
                lon_widget.disabled = True
            else:
                state_widget.disabled = True
                lat_widget.disabled = False
                lon_widget.disabled = False

        input_method.observe(toggle_inputs, names='value')

        # Other widgets
        crop_widget = widgets.Dropdown(options=list(CROP_INFO.keys()), value='Rice', description='Crop:')
        area_widget = widgets.FloatText(value=5.0, description='Area (ha):', step=0.1, min=0.1)
        irrigation_widget = widgets.IntSlider(value=10, min=0, max=25, description='Irrigation:')
        fertilizer_widget = widgets.IntSlider(value=200, min=0, max=600, description='Fertilizer:')
        pesticide_widget = widgets.IntSlider(value=3, min=0, max=12, description='Pesticide:')

        predict_button = widgets.Button(description='Predict Yield', button_style='success')
        output = widgets.Output()
        map_output = widgets.Output()

        # Layout
        location_box = widgets.VBox([input_method, state_widget, lat_widget, lon_widget])
        crop_area_box = widgets.HBox([crop_widget, area_widget])
        inputs_box = widgets.VBox([irrigation_widget, fertilizer_widget, pesticide_widget])

        interface = widgets.VBox([
            widgets.HTML("<h2>Smart Farm AI - Interactive Predictor</h2>"),
            location_box, crop_area_box, inputs_box, predict_button, output, map_output
        ])

        # Global system reference
        global interactive_system
        interactive_system = None

        def on_predict_click(b):
            with output:
                clear_output()
                print("Making prediction...")

                try:
                    global interactive_system
                    if interactive_system is None:
                        print("Training model (please wait)...")
                        interactive_system = CropYieldPredictionSystem()
                        df = interactive_system.generate_training_data(3000)
                        processed_df = interactive_system.preprocess_data(df)
                        interactive_system.train_models(processed_df)

                    # Get current lat/lon
                    current_lat = lat_widget.value if input_method.value == 'coord' else STATES[state_widget.value]['lat']
                    current_lon = lon_widget.value if input_method.value == 'coord' else STATES[state_widget.value]['lon']

                    # Make prediction
                    result = interactive_system.predict_yield(
                        current_lat, current_lon, crop_widget.value,
                        area_widget.value, irrigation_widget.value,
                        fertilizer_widget.value, pesticide_widget.value
                    )

                    # Generate recommendations and visualize
                    recommendations = interactive_system.generate_recommendations(result)
                    interactive_system.visualize_results(result, recommendations)

                except Exception as e:
                    print(f"Error: {e}")

            # Display map in separate output
            with map_output:
                clear_output()
                try:
                    current_lat = lat_widget.value if input_method.value == 'coord' else STATES[state_widget.value]['lat']
                    current_lon = lon_widget.value if input_method.value == 'coord' else STATES[state_widget.value]['lon']
                    interactive_system.display_location_map(current_lat, current_lon, result)
                except:
                    pass

        predict_button.on_click(on_predict_click)
        return interface

    print("Interactive interface functions created!")

    # Display the interface
    print("\nInteractive Interface:")
    interface = create_interactive_predictor()
    display(interface)

else:
    print("Install ipywidgets for interactive interface: !pip install ipywidgets")

# ============================================================================
# CELL 8: Usage Examples and Instructions
# ============================================================================

print("\n" + "="*60)
print("SMART FARM AI - READY TO USE!")
print("="*60)

print("\nUsage Options:")
print("1. quick_predict() - Fast prediction with visualization (now supports state)")
print("2. run_complete_demo() - Full demonstration")
print("3. Interactive interface (above) - If widgets are available")

print("\nQuick Examples:")
print("# Quick prediction for Rice in Punjab (state)")
print("result = quick_predict(state='Punjab', crop='Rice', area=5.0)")
print()
print("# Quick prediction for Wheat with coordinates")
print("result = quick_predict(lat=26.8467, lon=80.9462, crop='Wheat', area=10.0)")
print()
print("# Run full demo")
print("system = run_complete_demo()")

print("\nSupported Crops:", ', '.join(CROP_INFO.keys()))
print("Supported States:", ', '.join(STATES.keys()))

print("\nConfiguration:")
print("- Update WEATHER_API_KEY for real weather data")
print("- Modify CROP_INFO for additional crops")
print("- Customize STATES dict for more locations")
print("- Maps require folium (already installed)")

print("\nNew Features:")
print("- State-based input with automatic lat/lon mapping")
print("- Interactive map visualization with prediction popup and farm area circle")
print("- Toggle between state and coordinate input in UI")

print("\n" + "="*60)
print("Ready to revolutionize agriculture with AI!")
print("="*60)

✅ All libraries installed and imported successfully!
🌾 Smart Farm AI - Crop Yield Prediction System
🚀 Ready to predict crop yields using AI and real-time data!
⚙️ Configuration loaded successfully!
🌱 Supported crops: Rice, Wheat, Corn, Soybean, Cotton
🌍 Supported regions: North, South, East, West, Central
🏛️ Supported states: Punjab, Haryana, Uttar Pradesh, Rajasthan, Gujarat, Maharashtra, Madhya Pradesh, Bihar, West Bengal, Odisha, Andhra Pradesh, Karnataka, Tamil Nadu, Kerala, Telangana
🌤️ Weather integration class created successfully!
🌍 Soil integration class created successfully!
Complete Crop Yield Prediction System class created!
Interactive interface functions created!

Interactive Interface:



SMART FARM AI - READY TO USE!

Usage Options:
1. quick_predict() - Fast prediction with visualization (now supports state)
2. run_complete_demo() - Full demonstration
3. Interactive interface (above) - If widgets are available

Quick Examples:
# Quick prediction for Rice in Punjab (state)
result = quick_predict(state='Punjab', crop='Rice', area=5.0)

# Quick prediction for Wheat with coordinates
result = quick_predict(lat=26.8467, lon=80.9462, crop='Wheat', area=10.0)

# Run full demo
system = run_complete_demo()

Supported Crops: Rice, Wheat, Corn, Soybean, Cotton
Supported States: Punjab, Haryana, Uttar Pradesh, Rajasthan, Gujarat, Maharashtra, Madhya Pradesh, Bihar, West Bengal, Odisha, Andhra Pradesh, Karnataka, Tamil Nadu, Kerala, Telangana

Configuration:
- Update WEATHER_API_KEY for real weather data
- Modify CROP_INFO for additional crops
- Customize STATES dict for more locations
- Maps require folium (already installed)

New Features:
- State-based input with automatic lat/l